In [ ]:
!pip install pandas
!pip install scikit-learn
!pip install xgboost

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Loading the data

First I looked at what different tables were avaliable to us to try and understand what is relevent

In [2]:
#First create a connection
connection = sqlite3.connect('playerdata.sqlite')

# Then query the sqlite_master table to get information about all the tables
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the query
tables = connection.execute(tables_query).fetchall()

# Print the table names
for table in tables:
    print(table[0])

sqlite_sequence
Player_Attributes
Player
Match
League
Country
Team
Team_Attributes


Then with this information I decided to analyse the head of each table to recognise the features and types of data in each table. Did this by storing all the table names and looping through each one to displaying the contents. 

In [3]:
table_names = ['sqlite_sequence','Player_Attributes', 'Player', 'Match', 'League', 'Country', 'Team', 'Team_Attributes']

# Loop through each table and display first few rows
for table_name in table_names:
    # Query the table to get all rows and columns
    table_query = f"SELECT * FROM {table_name};"

    # Read the table data into a Pandas DataFrame
    table_data = pd.read_sql_query(table_query, connection)

    # Display the structure (columns) of the table
    #print(f"\nStructure of the '{table_name}' table:")
    #print(table_data.dtypes)

    # Display the first few rows of the table
    print(f"\nFirst few rows of the '{table_name}' table:")
    print(table_data.head(6))


First few rows of the 'sqlite_sequence' table:
                name     seq
0               Team  103916
1            Country   51958
2             League   51958
3              Match   51958
4             Player   11075
5  Player_Attributes  183978

First few rows of the 'Player_Attributes' table:
   id  player_fifa_api_id  player_api_id                 date  overall_rating  \
0   1              218353         505942  2016-02-18 00:00:00            67.0   
1   2              218353         505942  2015-11-19 00:00:00            67.0   
2   3              218353         505942  2015-09-21 00:00:00            62.0   
3   4              218353         505942  2015-03-20 00:00:00            61.0   
4   5              218353         505942  2007-02-22 00:00:00            61.0   
5   6              189615         155782  2016-04-21 00:00:00            74.0   

   potential preferred_foot attacking_work_rate defensive_work_rate  crossing  \
0       71.0          right              medium   

In [4]:
table_names = ['sqlite_sequence','Player_Attributes', 'Player', 'Match', 'League', 'Country', 'Team', 'Team_Attributes']

for table_name in table_names:
    table_data = pd.read_sql_query(f"SELECT * FROM {table_name};", connection)
    print(f"Statistical Summary of the '{table_name}' table:")
    print(table_data.describe())

Statistical Summary of the 'sqlite_sequence' table:
                 seq
count       7.000000
mean    65185.857143
std     62082.942398
min      1458.000000
25%     31516.500000
50%     51958.000000
75%     77937.000000
max    183978.000000
Statistical Summary of the 'Player_Attributes' table:
                 id  player_fifa_api_id  player_api_id  overall_rating  \
count  183978.00000       183978.000000  183978.000000   183142.000000   
mean    91989.50000       165671.524291  135900.617324       68.600015   
std     53110.01825        53851.094769  136927.840510        7.041139   
min         1.00000            2.000000    2625.000000       33.000000   
25%     45995.25000       155798.000000   34763.000000       64.000000   
50%     91989.50000       183488.000000   77741.000000       69.000000   
75%    137983.75000       199848.000000  191080.000000       73.000000   
max    183978.00000       234141.000000  750584.000000       94.000000   

           potential       crossing   

# Table Analysis for Dataset

This is the insight i got from the head of each table

- **sqlite_sequence**: 
  - Metadata table providing sequence numbers for other tables. 
  - Not directly relevant to performance analysis.

- **Player_Attributes**: 
  - Contains 42 attributes related to players' skills and abilities. 
  - Very useful for understanding and predicting player performance.

- **Player**: 
  - Provides player information like name, birthday most of which not relevant to performance. 
  - Derived metrics such as age and potentially the physical attributes (height, weight) could be valuable for analyzing physical aspects of performance.

- **Match**: 
  - Focuses on team performance in matches. 
  - Contextual understanding of player attributes during matches might offer insights.

- **League**: 
  - Offers league-specific contextual information. 
  - Useful for understanding performance trends across different leagues.

- **Country**: 
  - Similar to League, provides contextual data but not directly related to individual performance.

- **Team**: 
  - Contains basic information about teams. 
  - Team context can be influential in a player's performance.

- **Team_Attributes**: 
  - Focuses on team-level characteristics. 
  - More generalized attributes, indirectly related to individual player performance.

# Critical reflection

With this information, I wish to focus on Player_Attributes first to initate a base level prediction then if successfull can try and gain other insight that can further contextualise the predictions like what type of team they play for and what county and league they play in. Bellow is a clearer view of Player_Attributes most of which hidden due to size

In [ ]:
player_data = pd.read_sql_query("SELECT * FROM Player_Attributes", connection)
player_data.head().T

# Feature Analysis

After transposing I conducted a manual review of all the features and first separate the them into categorical and numerical features. 

#### Target attributes (What we predict)

'overall_rating' 



#### **Categorical attributes** - 
'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'

#### **Numerical attributes** - 
'potential', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes'

'id', 'player_fifa_api_id', 'player_api_id', and 'date' **NOT INCLUDED** as not useful in analysis

## Categorical attribute analysis

Now i will try and better understand what the attributes contain starting with the categorical attributes

In [ ]:
print(player_data['preferred_foot'].unique())
print(player_data['attacking_work_rate'].unique())
print(player_data['defensive_work_rate'].unique())

**Preferred Foot** - The values are 'right', 'left', and None. This is straightforward, representing the dominant foot of the players. The None value could indicate missing data.

**Attacking Work Rate** - Shows a range of values including 'medium', 'high', 'low', and some irregular entries like 'None', 'le', 'norm', 'stoc', 'y'. The expected values ('low', 'medium', 'high') represent a player's attacking work rate, but the irregular entries suggest data quality issues that need to be addressed.

**Defensive Work Rate** - Similar to attacking work rate, it includes 'low', 'medium', 'high', and a series of other values like '_0', '5', 'ean', etc. These additional values are likely to be errors or misentries.

<u>These will have to be cleaned and also one-hot encoded to numerical values in order to be trained in our model.</u>

## Numerical attribute analysis 


In [ ]:
numerical_attributes = ['potential', 'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression', 'interceptions', 'positioning', 'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes']

target_attributes = ['overall_rating']

for col in numerical_attributes:
    # Calculating basic statistics
    min_value = player_data[col].min()
    max_value = player_data[col].max()
    mean_value = player_data[col].mean()
    null_count = player_data[col].isnull().sum()

    # Displaying the statistics
    print(f"Analysis for {col}:")
    print(f"  - Min: {min_value}")
    print(f"  - Max: {max_value}")
    print(f"  - Mean: {mean_value}")
    print(f"  - Null Values: {null_count}")
    print("")


After analysing the numerical categories. It is clear that all the max and min values that are non null are between 0-100 which suggests that the ratings are similar to FIFA player scores as they are also from 0-100. 

### Potential Issue
It's important to note that while these ratings provide a standardized way to evaluate player skills, they might not directly translate to all football environments, especially at lower levels where such detailed assessments may not be available. Coaches and teams with limited resources might face challenges in precisely rating players using these metrics. However, these attributes still serve as a valuable guide and benchmark for player evaluation, even if the exact numerical values are not always available or directly applicable.

eg. will a coach know what a 60/100 strength is and how would he standardise this judgement to be similar to those that collected the training data.

### Proposed solutions
**Consistency**: Even if coaches rate players differently than the original FIFA metrics, maintaining a consistent approach is key. The model can still effectively track changes in player performance over time which is the main reason for the model anyway the actual number doesnt matter.

**Relative Performance Assessment**: Coaches can use the model to assess relative improvements or declines in a player's performance, which is valuable for training and team strategies.

**Customized Interpretation**: Coaches can interpret the ratings in a way that aligns with their team's specific needs and playing style, making the data relevant to their unique context. As long as they are consistent.


# Feature contextualisation and importance

## <u>Categorical Attributes:</u>

**Preferred Foot**: Does not significantly impact the overall rating however may be used for understanding position-specific attributes. 

**Attacking Work Rate**: Reflects a player's involvement in offensive play. Important for forwards and attacking midfielders, but its subjective nature can make it less reliable.

**Defensive Work Rate**: Same as attacking, essential for defensive players, yet subjective. It indicates a player's commitment to defense but doesn't always correlate directly with defensive skill.

## <u>Numerical Attributes:</u>

### Key Attributes for Performance:

**Ball Skills**: 'Dribbling', 'Ball Control'.

**Defensive Skills**: 'Marking', 'Standing Tackle', 'Sliding Tackle', 'Interceptions'.

**Physical Attributes**: 'Strength', 'Stamina', 'Jumping', 'Balance', 'Agility', 'Acceleration', 'Sprint Speed'.

**Mental Attributes**: 'Reactions', 'Aggression', 'Positioning', 'Vision'.

**Shooting Skills**: 'Finishing', 'Long Shots', 'Shot Power', 'Volleys'.

**Passing Skills**: 'Short Passing', 'Long Passing', 'Crossing'.

**Set-Piece Skills**: 'Free Kick Accuracy', 'Penalties'.

**Goalkeeping Skills**: 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes'.

### Attributes with Limited Impact:

**Potential**: Reflects a player's future capabilities, which can be speculative and hard to measure accurately.

**Curve**: Does not directly influence the match or a player's immediate performance

# Conclusion from Data Analysis:

The data analysis phase started with the exploration of the entire data set. Then able to discover the relevant parts to analyse. After that we revealed the intricate details of player attributes, offering a nuanced view of player skills and abilities. While some attributes like 'Dribbling' and 'Finishing' are pivotal for performance prediction, others such as 'Potential' or 'Preferred Foot' showed limited direct impact. The presence of irregular entries and null values in various attributes highlighted the need for comprehensive data cleaning and preprocessing.

# Approach for Preprocessing Stage:
As we transition to the preprocessing stage, our focus will be on refining the dataset to enhance the model's accuracy and relevance. This includes:

## Data Cleaning: 
Addressing irregular and missing values, especially in categorical data, to ensure data integrity.
## Feature Selection: 
Based on the critical analysis, less impactful features may be reconsidered for exclusion to streamline the model.
## Data Encoding and Scaling: 
Transforming categorical data into a format suitable for machine learning models and normalizing numerical data for uniformity.
## Preparing for Model Building: 
Ensuring the dataset is well-structured and optimized for the upcoming model training phase.